In [1]:
import os,sys,inspect
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil
import json

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
testname="Тест СКО"
display(Markdown('''### ░░░▄▀▀▀▄░ГУСЬ░

▄███▀░◐░░░▌░░░░░░░

░░░░▌░░░░░▐░░░░░░░

░░░░▐░░░░░▐░░░░░░░

░░░░▌░░░░░▐▄▄░░░░░

░░░░▌░░░░▄▀▒▒▀▀▀▀▄

░░░▐░░░░▐▒▒▒▒▒▒▒▒▀▀▄

░░░▐░░░░▐▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░▀▄░░░░▀▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░░░▀▄▄▄▄▄█▄▄▄▄▄▄▄▄▄▄▄▀▄

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░▄▄▌▌▄▌▌░░░░░ '''))
display(f'{testname}')

### ░░░▄▀▀▀▄░ГУСЬ░

▄███▀░◐░░░▌░░░░░░░

░░░░▌░░░░░▐░░░░░░░

░░░░▐░░░░░▐░░░░░░░

░░░░▌░░░░░▐▄▄░░░░░

░░░░▌░░░░▄▀▒▒▀▀▀▀▄

░░░▐░░░░▐▒▒▒▒▒▒▒▒▀▀▄

░░░▐░░░░▐▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░▀▄░░░░▀▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░░░▀▄▄▄▄▄█▄▄▄▄▄▄▄▄▄▄▄▀▄

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░▄▄▌▌▄▌▌░░░░░ 

'Тест СКО'

In [2]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)


FindDevice(children=(Dropdown(description='Выберите гуся:', layout=Layout(width='max-content'), options={' SN5…

In [3]:
button = box.select_device_continue_button

In [ ]:
get_ipython().run_line_magic('blockrun', 'button')

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
get_ipython().run_line_magic('blockrun', 'button1')

In [ ]:
inst = box.selected_device

In [ ]:
inst.write("SYST:PRES")

In [ ]:
result_config =  box.config

In [ ]:
def HZ_convert(pref, hertz):
    try:
        hertz = int(hertz)
    except ValueError:
        return -1
    if pref == "k":
        return hertz * 1000
    if pref == "M":
        return hertz * 1000000
    if pref == "G":
        return hertz * 1000000000
    return -1


# функция однократного россчерка
def one_scan(ch_num):
    inst.write('TRIG:SOUR BUS')  # устанавливаем источник триггера на шину
    inst.write(f"INIT{ch_num}:CONT OFF")  # включаем повторный режим запуска триггера
    # переходим в состояние ожидания триггера
    inst.write('INIT')
    inst.write('TRIG:SING')  # выполняем россчерк
    inst.query('*OPC?')

In [ ]:
def sko_algorithm(bwid, att, sko_data): # Прогонка по одному набору IF - ATT
    trace_count = int(inst.query("SERV:PORT:COUN?"))
    
    # Устанавливаем фильтр ПЧ 300 или 3 кГц
    inst.write(f'SENS1:BWID {f"{bwid}"}')

    # вкл. Аттенюатор 10, 30 или 50 дБ
    inst.write('SERVice:RFCTL:POWer:STATe 1')
    inst.write(f'SERVice:RFCTL:POWer:ATT {att}')

    # Частотный план
    inst.write('SERVice:SWEep:FREQuency:FACTory')
    if (f'{att}' not in sko_data['atthenuator']):
        sko_data['atthenuator'].update({f'{att}':{'IF':{}}})
    sko_data['atthenuator'][f'{att}']['IF'].update({f'{bwid}' : {'trace':{}}})
    

    sko_data['atthenuator'][f'{att}']['IF'][f'{bwid}']['trace'].update({'otnosit' : {'max_value':0}})
    sko_data['atthenuator'][f'{att}']['IF'][f'{bwid}']['trace'].update({'absolute' : {'max_value':0}})          

    for i in range(1, trace_count+1): # Установка трасс
        inst.write('CALC1:PAR:COUN 3')
        inst.write(f'CALC1:PAR1:DEF S{i}{i}') #S
        inst.write(f'CALC1:PAR2:DEF R{i}')
        inst.write(f'CALC1:PAR2:SPOR {i}') # R
        inst.write(f'CALC1:PAR3:DEF T{i}')
        inst.write(f'CALC1:PAR3:SPOR {i}') # T
        one_scan(1)
        for k in range(1, 4): # нормализация
            inst.write(f'CALC:PAR{k}:SEL')
            inst.write('CALC:MATH:MEM')
            inst.write('CALC:MATH:FUNC DIV')
        one_scan(1)

        for k in range(1,4): # запись SKO
            inst.write(f'CALC:PAR{k}:SEL')
            inst.write('CALC1:MST ON')
            mst_data = inst.query(f'CALC1:MST:DATA?').split(',')
#             print(f'this is data {mst_data}')
            # format(float(mst_data[1]), '.8f')
            if k == 1:
               sko_data['atthenuator'][f'{att}']['IF'][f'{bwid}']['trace']['otnosit'][f'S{i}{i}'] = float(mst_data[1])
#                print('S')
            elif k == 2:
               sko_data['atthenuator'][f'{att}']['IF'][f'{bwid}']['trace']['absolute'][f'R{i}{i}'] = float(mst_data[1])
#                print('R')
            else:
               sko_data['atthenuator'][f'{att}']['IF'][f'{bwid}']['trace']['absolute'][f'T{i}{i}'] = float(mst_data[1])
        pbar.update()
    sko_data['atthenuator'][f'{att}']['IF'][f'{bwid}']['trace']['otnosit']['max_value'] = max(sko_data['atthenuator'][f'{att}']['IF'][f'{bwid}']['trace']['otnosit'].values())
    sko_data['atthenuator'][f'{att}']['IF'][f'{bwid}']['trace']['absolute']['max_value'] = max(sko_data['atthenuator'][f'{att}']['IF'][f'{bwid}']['trace']['absolute'].values())
    
    return(sko_data)

In [ ]:
def sko_test(bwids, attitudes): # Прогонка по всем параметрам
    result = {'atthenuator':{}}
    for att in attitudes:
        for bwid in bwids:
            result = sko_algorithm(int(bwid), int(att), result)
    return result
    

In [ ]:
def get_iff(iff_list):
    iff = []
    arr = iff_list.split(',')
    for i in range(len(arr)):
        arr1 = arr[i].split(' ')
        num = arr1[0]
        pref = arr1[1]
        iff.append(HZ_convert(pref, num))
    return iff

In [ ]:
config = box.config
iff = get_iff(config['DEVICE']['filter_IF'])
mass = (config['DEVICE']['athenuator']).split(',')
trace_count = int(inst.query('SERV:PORT:COUN?'))
pbar = tqdm_notebook(total=trace_count*len(mass)*len(iff))
result = sko_test(iff, mass)

In [ ]:
def html_table(data, acces):
    htmlstr = '''<html>
        <style>
        table, th, td {
          border:1px solid black;
        }
        </style>
        <body>
            <table style="width:100%; margin-left:auto;margin-right:auto;">
                <tr>
                    <th>
                        Канал
                    </th>
                    <th>
                        ПЧ, Гц
                    </th>
                    <th>
                        Аттюнеатор, дБ
                    </th>
                    <th>
                        Измерение, дБ
                    </th>
                    <th>
                        Максимум, дБ
                    </th>
                    <th>
                        Допуск, дБ
                    </th>
                </tr>
            '''
    for i in data["atthenuator"]:
        for j in data["atthenuator"][i]["IF"]:   # 300/3
            for k in data["atthenuator"][i]["IF"][j]["trace"]:  # abs/otn
                count = 0
                channel_arr = []
                channel_arr_sko = []
                for g in data["atthenuator"][i]["IF"][j]["trace"][k]:   # T, R, S, max_value
                    max_value = format(float(data["atthenuator"][i]["IF"][j]["trace"][k]["max_value"]), '.8f')
                    if count != 0:  # т.к. 0 индекс max value
                        channel_arr_sko.append(format(float(data["atthenuator"][i]["IF"][j]["trace"][k][g]), '.8f'))  # запись СКО
                        channel_arr.append(g)
                    count += 1
                    if count == (len(data["atthenuator"][i]["IF"][j]["trace"][k])):     # если достали все данные одного прохода
                        # print(data["atthenuator"][i]["IF"][j]["trace"][k])
                        # print(len(data["atthenuator"][i]["IF"][j]["trace"][k]))
#                         acces = 0
#                         if k == "otnosit":
#                             acces = access_otn
#                         else:
#                             acces = access_abs
                        trace_num = len(channel_arr)

                        if float(channel_arr_sko[0]) > float(acces):    # запись строчек
                            htmlstr += f'''<tr><td>{channel_arr[0]}</td><td rowspan = {trace_num}>{j} </td><td rowspan = {trace_num}>{i}</td><td style="background-color:#FFFF00">{channel_arr_sko[0]}</td><td rowspan = {trace_num}>{max_value}</td><td rowspan = {trace_num}>{acces}</td></tr>'''
                        else:
                            htmlstr += f'''<tr><td >{channel_arr[0]}</td><td rowspan = {trace_num}>{j}</td><td rowspan = {trace_num}>{i}</td><td>{channel_arr_sko[0]}</td><td rowspan = {trace_num}>{max_value}</td><td rowspan = {trace_num}>{acces}</td></tr>'''
                        for traces in range(1, trace_num):
                            if float(channel_arr_sko[traces]) > float(acces):  # запись строчечк
                                htmlstr += f'''<tr><td>{channel_arr[traces]}</td><td style="background-color:#FFFF00">{channel_arr_sko[traces]}</td></tr>'''
                            else:
                                htmlstr += f'''<tr><td>{channel_arr[traces]}</td><td>{channel_arr_sko[traces]}</td></tr>'''

                        # if float(max_value) > float(acces):    # запись строчечк
                        #     htmlstr += f'''<tr><td>{channel_arr}</td><td>{j}</td><td>{i}</td><td style="background-color:#FFFF00">{channel_arr_sko}</td><td>{max_value}</td><td>{acces}</td></tr>'''
                        # else:
                        #     htmlstr += f'''<tr><td>{channel_arr}</td><td>{j}</td><td>{i}</td><td>{channel_arr_sko}</td><td>{max_value}</td><td>{acces}</td></tr>'''

    htmlstr += '''</table>
        </body>
    </html>'''
    return htmlstr

In [ ]:
mark=Markdown('### 🦢Укажите допуск Дб')
text=widgets.Text(
        value='',
        placeholder='Допуск',
        description='Допуск',
        disabled=False)
button = widgets.Button(
    description='Continue',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='play'        
    )
display(mark,text,button)


In [ ]:
%blockrun button

In [ ]:

from IPython.core.display import display, HTML
file_data = html_table(result, float(text.value))
html_table=file_data


display(HTML(html_table))
my_button = SaveFilesButton(file_data)
my_button # This will display the button in the context of Jupyter Notebook